In [1]:
# import libs
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from bs4 import BeautifulSoup
import seaborn as sns
import plotly.express as px
import statsmodels.api as sm

In [ ]:
df = pd.read_csv('../IBRA_D.csv', sep=';') # dados baixados do site da b3 composição da carteira IBRAX

In [ ]:
df.rename(columns={"C�digo": "CODIGO", "A��o": "AÇAO"}, inplace=True)
df = df.iloc[:-2, :]

In [ ]:
df.tail()

In [ ]:
Symbols = df['CODIGO'] + '.SA' # junção para coleta dos dados dos ativos em yahoo finance

In [ ]:
Symbols

In [ ]:
data_inicial = datetime(2019, 12, 31) 
data_final = datetime.today()

In [ ]:
def pegar_dados(codigoAtivo):
    ativo = yf.download(codigoAtivo, start=data_inicial, end=data_final)
    return ativo

In [ ]:
tickers = [tick for tick in Symbols]
tickers

In [ ]:
dicionarioAtivos = {}

In [ ]:
for t in tickers:
    try:
        dicionarioAtivos[t] = pegar_dados(t)
    
    except:
        print(f'NÃO FOI FEITA A COLETA DO ATIVO : {t}')

In [ ]:
dicionarioAtivos

In [ ]:
retornos_ativos = pd.DataFrame()

In [ ]:
for chave in dicionarioAtivos.keys():
    try:
        data = pd.DataFrame(dicionarioAtivos[chave])
        retornos_ativos[chave] = data['Adj Close'].pct_change()

    except:
        print('NÃO FOI FEITO O CALCULOS DOS RETORNOS')


In [ ]:
retornos_ativos

In [ ]:
dicionarioDescontos = {}

In [ ]:
def calc_descontos(df):
    max_price = df['Adj Close'].describe()['75%']
    min_price = df['Adj Close'].iloc[-1]
    
    return(min_price - max_price) / max_price

In [ ]:
for tick in retornos_ativos.columns:
    dicionarioDescontos[tick] = calc_descontos(dicionarioAtivos[tick])

In [ ]:
dicionarioDescontos

In [ ]:
ativos_info = pd.DataFrame.from_dict(dicionarioDescontos, orient='index').reset_index()

In [ ]:
ativos_info

In [ ]:
ativos_info.columns = ["Ticker", "Desconto"]

In [ ]:
ativos_info

In [ ]:
# uso do desvio padrão para calculo de risco
stdlist = []

In [ ]:
for tick in ativos_info['Ticker']:
    stdlist.append(retornos_ativos[tick].std())

In [ ]:
ativos_info['Risco'] = stdlist

In [ ]:
ativos_info

In [ ]:
ultimo_preço = [dicionarioAtivos[tick]['Adj Close'].iloc[-1] for tick in ativos_info['Ticker']]

In [ ]:
ativos_info['ultimo_preço'] = ultimo_preço

In [ ]:
ativos_info # usar dados para analise mais profunda

In [ ]:
index_names = ativos_info[ativos_info['Ticker'] == 'BPAN4'].index
ativos_info.drop(index_names, inplace=True)

In [ ]:
index_names = ativos_info[ativos_info['Ticker'] == 'AMER3.SA'].index
ativos_info.drop(index_names, inplace=True)

In [ ]:
index_names = ativos_info[ativos_info['Ticker'] == 'ITUB3.SA'].index
ativos_info.drop(index_names, inplace=True)

In [ ]:
index_names = ativos_info[ativos_info['Ticker'] == 'ITUB4.SA'].index
ativos_info.drop(index_names, inplace=True)

In [ ]:
ativos_info_top20 = ativos_info.sort_values(by='Desconto')[:21]

In [ ]:
ativos_info_top20

In [ ]:
# ordenar valores desconto
ativos_info.sort_values(by = 'Desconto')

In [ ]:
# ordenar valores desconto
ativos_info.sort_values(by = 'Risco')

In [ ]:
fig = px.scatter(ativos_info, x='Risco', y='Desconto', size='ultimo_preço', hover_name='Ticker', log_x=True)
fig.show()

In [ ]:
fig2 = px.scatter(ativos_info_top20, x='Risco', y='Desconto', size='ultimo_preço', hover_name='Ticker', log_x=True)
fig.show()